In [2]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [3]:
import pandas as pd

In [28]:
#sample

#rdd1_lst = []
#file = sc.textFile('file:///home/ubuntu/Ybigta/Project/bitcoin_project/data/sample.csv')
#rdd1_lst.append(file)
#rdd1_lst[0].take(5)

In [29]:
rdd1_lst = []
for idx in range(1, 15) :
    file = sc.textFile('file:///home/ubuntu/Ybigta/Project/bitcoin_project/data/data{}'.format(idx))
    rdd1_lst.append(file)
    
    #print(idx, ":",file.count())

rdd1_lst[0].take(5)

['timestamp,amount,price',
 '2017-02-13 10:33:09,0.39,985.5',
 '2017-02-13 10:33:43,2.0,985.5',
 '2017-02-13 10:34:07,0.139,985.5',
 '2017-02-13 10:34:09,0.03,985.5']

In [30]:
rdd2_lst = []
for idx, rdd in enumerate(rdd1_lst) :
    temp = rdd.filter(lambda line : "price" not in line)
    rdd2_lst.append(temp)
    #rdd1_lst[idx] = False
    
    #print(idx, ":",temp.count())
    
rdd2_lst[0].take(5)

['2017-02-13 10:33:09,0.39,985.5',
 '2017-02-13 10:33:43,2.0,985.5',
 '2017-02-13 10:34:07,0.139,985.5',
 '2017-02-13 10:34:09,0.03,985.5',
 '2017-02-13 10:34:35,-6.98,985.2']

In [39]:
def mapper(line, mode) :
    
    if mode == "4h" : 
        splited = line.split(",")
    
        date, time = splited[0].split()
        year, month, day = date.split("-")
        time = int(time.split(":")[0])
    
        amount = float(splited[1])
        price = float(splited[2])
        
        for idx in range(0, 21, 4) :
            if idx <= time and time < idx+4 :
                time = idx
                break
        
        if time < 10 :
            time = "0" + str(time)
        else :
            time = str(time)
            
        key = date + " " + time + ":00:00"
        value =  (amount, amount, price, price, price, price, price, 1, price*abs(amount), price*amount)
        return (key, value)
        
    elif mode == "6h" :
        splited = line.split(",")
    
        date, time = splited[0].split()
        year, month, day = date.split("-")
        time = int(time.split(":")[0])
    
        amount = float(splited[1])
        price = float(splited[2])
        
        for idx in range(0, 19, 6) :
            if idx <= time and time < idx+6 :
                time = idx
                break
                
        if time < 10 :
            time = "0" + str(time)
        else :
            time = str(time)
                
        key = date + " " + time + ":00:00"
        value =  (amount, amount, price, price, price, price, price, 1, price*abs(amount), price*amount)
        return (key, value)
        
    elif mode == "12h" :
        input_key = line[0]
        input_value = line[1]
        
        date, time = input_key.split()
        year, month, day = date.split("-")
        time = int(time.split(":")[0])
        
        for idx in range(0, 13, 12) :
            if idx <= time and time < idx+12 :
                time = idx
                break
                
        if time < 10 :
            time = "0" + str(time)
        else :
            time = str(time)
        
        key = date + " " + time + ":00:00"
        value = input_value
        return (key, value)
        
    elif mode == "1d" :
        input_key = line[0]
        input_value = line[1]
        
        date, time = input_key.split()
        year, month, day = date.split("-")
        time = int(time.split(":")[0])
                   
        key = date + " 00:00:00"
        value = input_value
        return (key, value)
    
    elif mode == "7d" :
        input_key = line[0]
        input_value = line[1]
        
        date, time = input_key.split()
        year, month, day = date.split("-")
        day = int(day)
        
        for idx in range(1, 32, 7) :
            if idx <= day and day < idx+7 :
                day = idx
                
                if day >= 22 :
                    day = 22
                
                
        if day < 10 :
            day = "0" + str(day)
        else :
            day = str(day)
        
        key = year + "-" + month + "-" + day + " 00:00:00"
        value = input_value
        return (key, value)
        
        
    elif mode == "1m" :
        input_key = line[0]
        input_value = line[1]
        
        date, time = input_key.split()
        year, month, day = date.split("-")
                   
        key = year + "-" + month + "-00 00:00:00"
        value = input_value
        return (key, value)

In [40]:
def reducer(x,y) :
    #부호
    sign = x[0] + y[0]
    
    #거래량
    amount = abs(x[1]) + abs(y[1])
    
    #시가
    start_price = x[2]
    
    #종가
    end_price = y[3]
    
    #최고가격
    max_price = max(x[4], y[4])
    
    #평균가격
    avg_price = (x[5]*x[7] + y[5]*y[7]) / (x[7]+y[7])
    
    #최저가격
    min_price = min(x[6], y[6])
    
    #거래횟수
    count = x[7] + y[7]
    
    #거래금액
    total_money = x[8] + y[8]
    
    #시총증감
    market_cap = x[9] + y[9]
    
    return (sign, amount, start_price, end_price, max_price, avg_price, min_price, count, total_money, market_cap)

In [41]:
rdd3_lst = []

for idx, rdd in enumerate(rdd2_lst) :
    rdd_4h = rdd.map(lambda line : mapper(line, "4h")).reduceByKey(lambda x,y : reducer(x,y))
    rdd_6h = rdd.map(lambda line : mapper(line, "6h")).reduceByKey(lambda x,y : reducer(x,y))
    rdd_12h = rdd_6h.map(lambda line : mapper(line, "12h")).reduceByKey(lambda x,y : reducer(x,y))
    rdd_1d = rdd_12h.map(lambda line : mapper(line, "1d")).reduceByKey(lambda x,y : reducer(x,y))
    rdd_7d = rdd_1d.map(lambda line : mapper(line, "7d")).reduceByKey(lambda x,y : reducer(x,y))
    rdd_1m = rdd_7d.map(lambda line : mapper(line, "1m")).reduceByKey(lambda x,y : reducer(x,y))
    
    rdd3_lst.append([rdd_4h,rdd_6h,rdd_12h,rdd_1d,rdd_7d,rdd_1m])
    #rdd2_lst[idx] = False
    print(idx)

0
1
2
3
4
5
6
7
8
9
10
11
12
13


In [42]:
rdd3_lst[0][0].take(5)

[('2017-02-13 12:00:00',
  (276.33046248999995,
   1071.758302249999,
   990.91,
   992.79,
   992.9,
   990.9003986135178,
   986.9,
   1154,
   1062141.8600795693,
   274193.037093254)),
 ('2017-02-13 16:00:00',
  (0.2495288900000986,
   783.3258473099978,
   992.88,
   995.39,
   996.88,
   994.0527252747223,
   989.64,
   910,
   778133.6220460183,
   528.1515629759015)),
 ('2017-02-13 20:00:00',
  (175.93711095999967,
   967.4575365999985,
   995.06,
   996.5,
   999.9,
   996.7483828775253,
   993.2,
   841,
   964173.2125144713,
   175705.3945342737)),
 ('2017-02-14 00:00:00',
  (205.21141930999954,
   861.0315760899884,
   996.52,
   996.02,
   999.05,
   997.3644668989526,
   995.13,
   1435,
   858520.7923367467,
   204692.93388866947)),
 ('2017-02-14 04:00:00',
  (310.7467760500008,
   2230.141009069999,
   996.0,
   1004.0,
   1007.8,
   1001.056534541334,
   988.6,
   1766,
   2232592.2833704804,
   313053.46687578387))]

In [43]:
rdd4_lst = list(zip(*rdd3_lst))
print(len(rdd4_lst), len(rdd4_lst[0]))

6 14


In [44]:
rdd5_lst = []

for lst in rdd4_lst :
    result = sc.parallelize(range(0))
    
    for rdd in lst :
        result = result.union(rdd)
    result = result.reduceByKey(lambda x,y : reducer(x,y))
    
    print("check!")
    rdd5_lst.append(result)

check!
check!
check!
check!
check!
check!


In [45]:
print(len(rdd5_lst))
print("")
#rdd5_lst[0].take(5)
#rdd5_lst[1].take(5)
#rdd5_lst[2].take(5)
#rdd5_lst[3].take(5)
#rdd5_lst[4].take(5)
#rdd5_lst[5].take(5)

6



In [46]:
rdd5_lst[0].saveAsTextFile("file:///home/ubuntu/Ybigta/Project/bitcoin_project/output/output_4h")
rdd5_lst[1].saveAsTextFile("file:///home/ubuntu/Ybigta/Project/bitcoin_project/output/output_6h")
rdd5_lst[2].saveAsTextFile("file:///home/ubuntu/Ybigta/Project/bitcoin_project/output/output_12h")
rdd5_lst[3].saveAsTextFile("file:///home/ubuntu/Ybigta/Project/bitcoin_project/output/output_1d")
rdd5_lst[4].saveAsTextFile("file:///home/ubuntu/Ybigta/Project/bitcoin_project/output/output_7d")
rdd5_lst[5].saveAsTextFile("file:///home/ubuntu/Ybigta/Project/bitcoin_project/output/output_1m")

In [47]:
print(rdd5_lst[0].count())
rdd_4h = rdd5_lst[0].collect()

2352


In [48]:
print(rdd5_lst[1].count())
rdd_6h = rdd5_lst[1].collect()

1569


In [49]:
print(rdd5_lst[2].count())
rdd_12h = rdd5_lst[2].collect()

785


In [50]:
print(rdd5_lst[3].count())
rdd_1d = rdd5_lst[3].collect()

393


In [51]:
print(rdd5_lst[4].count())
rdd_7d = rdd5_lst[4].collect()

53


In [52]:
print(rdd5_lst[5].count())
rdd_1m = rdd5_lst[5].collect()

14


In [53]:
df_4h = pd.DataFrame.from_items(rdd_4h).T
df_6h = pd.DataFrame.from_items(rdd_6h).T
df_12h = pd.DataFrame.from_items(rdd_12h).T
df_1d = pd.DataFrame.from_items(rdd_1d).T
df_7d = pd.DataFrame.from_items(rdd_7d).T
df_1m = pd.DataFrame.from_items(rdd_1m).T

In [54]:
df_4h.head(20)

,0,1,2,3,4,5,6,7,8,9
2017-02-21 00:00:00,-11.876237,2593.005813,1091.800000,1085.30,1093.6,1088.233494,1085.0,1657.0,2.822308e+06,-1.239793e+04
2017-02-25 16:00:00,-1141.220151,8066.598994,1162.000000,1153.80,1163.0,1144.554482,1131.3,4440.0,9.226222e+06,-1.304313e+06
2017-02-27 00:00:00,53.432637,1946.723810,1178.700000,1191.00,1194.7,1188.461102,1178.7,1851.0,2.314917e+06,6.391866e+04
2017-03-14 08:00:00,922.705853,1753.055167,1225.400000,1240.50,1240.6,1233.114932,1225.4,1768.0,2.164311e+06,1.140452e+06
2017-03-25 20:00:00,3002.950271,7883.683132,944.300000,972.17,981.0,959.081194,937.0,5253.0,7.584790e+06,2.879144e+06
2017-03-26 04:00:00,-50.305253,1908.518372,970.000000,957.60,971.0,962.186373,950.1,1905.0,1.833827e+06,-4.733523e+04
2017-03-28 20:00:00,358.017608,4312.384993,1026.100000,1044.70,1046.8,1036.723973,1023.5,3579.0,4.475587e+06,3.728824e+05
2017-05-03 04:00:00,-494.878967,2741.757969,1575.000000,1551.60,1575.0,1555.696140,1541.9,3886.0,4.266654e+06,-7.725109e+05
2017-05-14 04:00:00,-103.889531,1383.350170,1828.700000,1845.90,1845.9,1822.940708,1807.9,2007.0,2.521545e+06,-1.844860e+05
2017-05-14 08:00:00,-137.836150,2133.791462,1845.800000,1818.50,1852.4,1832.939561,1807.1,3051.0,3.911701e+06,-2.426329e+05


In [55]:
df_6h.head(20)

,0,1,2,3,4,5,6,7,8,9
2017-02-21 00:00:00,276.643856,3279.165321,1091.8,1089.200000,1093.6,1088.286291,1085.0,2006.0,3.569377e+06,3.014804e+05
2017-02-27 00:00:00,-370.452716,2792.653313,1178.7,1181.000000,1194.7,1186.743846,1170.0,2543.0,3.312711e+06,-4.355602e+05
2017-03-02 06:00:00,-163.397381,2801.166044,1219.1,1232.400000,1234.1,1227.296231,1219.0,2786.0,3.439547e+06,-2.004434e+05
2017-04-04 18:00:00,415.970999,2421.720161,1136.0,1145.800000,1145.9,1137.717210,1130.2,3254.0,2.756444e+06,4.756319e+05
2017-04-21 06:00:00,317.950996,1825.485196,1321.1,1326.900000,1333.1,1321.825522,1313.2,3209.0,2.414488e+06,4.242347e+05
2017-05-05 18:00:00,-1432.462645,8045.835604,1588.5,1545.100000,1592.2,1556.115126,1505.0,12839.0,1.249589e+07,-2.220248e+06
2017-05-20 06:00:00,88.325888,4604.635801,1994.0,1977.400000,1998.9,1986.224659,1968.0,5499.0,9.148230e+06,1.917582e+05
2017-05-28 06:00:00,453.903860,5921.945462,2039.3,2135.000000,2194.4,2120.636563,2029.6,10445.0,1.259967e+07,9.615685e+05
2017-06-08 00:00:00,415.107028,6731.212078,2644.1,2770.000000,2780.0,2665.617254,2527.0,8003.0,1.787039e+07,1.232984e+06
2017-06-09 06:00:00,-49.221913,1357.371569,2780.0,2802.000000,2812.3,2791.597785,2763.9,2799.0,3.791804e+06,-1.357122e+05


In [56]:
df_12h.head(20)

,0,1,2,3,4,5,6,7,8,9
2017-02-21 00:00:00,1882.661391,10648.014369,1091.80,1115.5,1116.000000,1097.472955,1085.00,5709.0,1.170651e+07,2.079660e+06
2017-02-27 00:00:00,-481.248612,4593.785623,1178.70,1181.9,1194.700000,1184.681059,1170.00,4081.0,5.440369e+06,-5.661558e+05
2017-06-08 00:00:00,86.604642,9719.605644,2644.10,2725.0,2780.000000,2698.144258,2527.00,13025.0,2.609633e+07,3.364199e+05
2017-09-01 12:00:00,132.408157,17061.426966,4788.30,4907.7,4936.900000,4835.420120,4765.80,31977.0,8.255594e+07,7.176746e+05
2017-09-19 12:00:00,-1126.096754,21954.774593,3988.50,3892.2,4040.000000,3947.948003,3848.80,30114.0,8.660914e+07,-4.377125e+06
2017-10-21 12:00:00,-411.122576,31943.936768,6113.00,6005.1,6199.900000,6062.360490,5862.80,41636.0,1.933675e+08,-2.156456e+06
2017-10-24 12:00:00,-6315.166799,26007.867732,5700.30,5510.0,5733.000000,5607.416710,5457.40,43647.0,1.454208e+08,-3.505715e+07
2018-01-05 00:00:00,1007.190599,25584.250683,15159.00,16104.0,16242.000000,15409.500033,14725.00,86659.0,3.973847e+08,1.701760e+07
2018-02-05 12:00:00,-8620.230576,91579.802226,7585.80,6946.6,7808.626334,7189.000922,6651.00,189614.0,6.568075e+08,-6.198263e+07
2018-02-08 00:00:00,-1472.684212,39241.527642,7985.10,7985.1,8619.400000,8131.349184,7565.00,89630.0,3.187517e+08,-1.130285e+07


In [57]:
df_1d.head(50)

,0,1,2,3,4,5,6,7,8,9
2017-02-21 00:00:00,4346.660382,23519.556898,1091.80,1129.60,1134.0,1108.885097,1085.000000,12689.0,2.609007e+07,4.848749e+06
2017-02-27 00:00:00,-795.890982,10511.772617,1178.70,1195.50,1204.0,1187.629119,1170.000000,8661.0,1.248197e+07,-9.333584e+05
2017-06-08 00:00:00,287.990379,16130.233263,2644.10,2781.50,2789.5,2710.492137,2527.000000,26529.0,4.351379e+07,9.136926e+05
2018-01-05 00:00:00,4220.761350,55310.606617,15159.00,16401.00,17105.0,15954.729912,14725.000000,189984.0,8.855082e+08,7.058091e+07
2018-02-08 00:00:00,-9366.846043,78240.484720,8469.50,7985.10,8648.9,8227.345097,7565.000000,174486.0,6.432578e+08,-7.681614e+07
2017-03-28 00:00:00,-193.043501,23852.316981,1065.50,1065.60,1068.9,1043.628296,1016.200000,21240.0,2.489142e+07,-1.698185e+05
2017-03-30 00:00:00,-625.186380,24461.675949,1034.10,1032.90,1053.5,1034.630409,1021.200000,37647.0,2.531013e+07,-6.348997e+05
2017-05-12 00:00:00,-3168.451039,20780.727352,1797.10,1814.10,1864.8,1788.213118,1700.200000,30705.0,3.711578e+07,-5.580888e+06
2017-05-18 00:00:00,1593.006405,12438.707358,1883.10,1941.50,1954.6,1900.125656,1861.000000,22650.0,2.369103e+07,3.074233e+06
2017-06-22 00:00:00,-2483.971776,14812.076383,2640.20,2640.20,2709.0,2647.578514,2554.800000,28850.0,3.919262e+07,-6.524041e+06


In [58]:
df_7d.head(50)

,0,1,2,3,4,5,6,7,8,9
2017-06-08 00:00:00,-7328.906973,159744.076458,2644.10,2394.3,3003.0,2688.490083,2280.000000,299474.0,4.262122e+08,-1.789768e+07
2018-02-08 00:00:00,-7478.460257,406257.059904,8679.10,8850.0,9489.0,8494.531674,7565.000000,1004289.0,3.447332e+09,-5.012858e+07
2017-06-22 00:00:00,-15402.606968,168680.480443,2672.80,2472.4,2726.0,2480.033207,2207.800000,297362.0,4.163183e+08,-3.757526e+07
2017-10-08 00:00:00,25002.268930,340083.256622,4611.80,5705.1,5920.0,5156.486934,4416.300000,488358.0,1.760093e+09,1.251464e+08
2018-01-22 00:00:00,-5891.894860,464731.773352,10151.00,10000.0,12190.0,10925.170493,9760.000000,1381697.0,5.051846e+09,-4.778784e+07
2017-08-08 00:00:00,2715.393766,214279.307446,3395.90,4160.8,4330.3,3712.119129,3235.100000,427435.0,7.996133e+08,1.337097e+07
2017-04-22 00:00:00,1463.925430,86738.503736,1327.00,1421.5,1449.7,1384.348244,1290.000000,147919.0,1.198508e+08,2.193533e+06
2017-09-08 00:00:00,-30072.742509,388005.508999,4613.60,4212.9,4694.9,4022.314292,3204.700000,492184.0,1.538283e+09,-1.124925e+08
2017-09-15 00:00:00,-10461.194666,458453.528331,3235.40,3596.7,4119.0,3671.419925,2980.400000,584393.0,1.668478e+09,-3.294919e+07
2017-02-08 00:00:00,924.036913,15285.133131,990.91,1011.6,1019.0,1004.422295,985.200000,14892.0,1.535681e+07,9.216794e+05


In [59]:
df_1m.head(20)

,0,1,2,3,4,5,6,7,8,9
2017-08-00 00:00:00,7766.072464,9.089799e+05,3395.9,4718.3,4749.9,3841.217940,2604.0,1737401.0,3.488218e+09,2.647708e+07
2017-11-00 00:00:00,-17297.725825,2.139144e+06,7370.0,8075.0,11441.0,7848.368922,5400.2,3886352.0,1.662470e+10,-8.640135e+07
2018-03-00 00:00:00,-15879.046475,5.667118e+05,9341.7,10997.0,11700.0,10066.452802,8342.0,1432981.0,5.643233e+09,-1.437478e+08
2018-02-00 00:00:00,-71454.954283,1.956508e+06,9181.0,10706.0,11788.0,9126.682497,6000.0,4598529.0,1.767686e+10,-5.806102e+08
2017-07-00 00:00:00,-30199.772889,9.093860e+05,2420.6,2781.0,2922.0,2468.756736,1826.2,1256318.0,2.249349e+09,-6.739870e+07
2017-06-00 00:00:00,-35330.590616,6.258366e+05,2404.7,2472.4,3003.0,2552.483379,2046.0,1083480.0,1.590285e+09,-8.711801e+07
2017-10-00 00:00:00,10599.715704,1.305159e+06,5834.9,5673.8,6481.5,5440.218818,4133.0,1966915.0,7.103492e+09,5.201460e+07
2017-09-00 00:00:00,-36414.532892,1.420370e+06,4613.6,3930.1,4970.0,4014.377328,2980.4,2002996.0,5.618896e+09,-1.314337e+08
2017-03-00 00:00:00,-24993.458466,9.861310e+05,1189.1,1115.9,1339.9,1108.189138,888.2,737527.0,1.094615e+09,-2.686917e+07
2017-05-00 00:00:00,-11507.693193,7.149715e+05,1435.3,2019.4,2690.0,1957.696915,1432.0,1172948.0,1.392532e+09,-2.439247e+07


In [61]:
df_4h.to_csv("output/csv/bitcoin_4h.csv")
df_6h.to_csv("output/csv/bitcoin_6h.csv")
df_12h.to_csv("output/csv/bitcoin_12h.csv")
df_1d.to_csv("output/csv/bitcoin_1d.csv")
df_7d.to_csv("output/csv/bitcoin_7d.csv")
df_1m.to_csv("output/csv/bitcoin_1m.csv")